<p>
  <a href="https://colab.research.google.com/github/ezhilvendhan/ecommerce-demo-gds-vertex-ai/blob/main/similarity.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

# Install Prerequisites
First off, you'll also need to install a few packages.

In [ ]:
%pip install --quiet --upgrade graphdatascience
%pip install --quiet google-cloud-storage
%pip install --quiet google.cloud.aiplatform

# Restart the Kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.  When you run this, you may get a notification that the kernel crashed.  You can disregard that.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

# Authenticate your Google Cloud Account
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must first authenticate.

Edit the variables below.  You can find the project ID in the Google Cloud Console.  The STORAGE_BUCKET is the name of a new bucket.  It must be globally unique.  It also needs to be all lower case.

In [16]:
# Edit this variable!
PROJECT_ID = 'neo4jbusinessdev'
REGION = 'us-central1'

In [14]:
import os
os.environ['GCLOUD_PROJECT'] = PROJECT_ID

In [15]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Load data to BigQuery

## Create Big Query Tables


In [9]:
import pandas_gbq
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
client.project

'neo4jbusinessdev'

#### Create Dataset if needed

In [18]:
from google.api_core.exceptions import NotFound
dataset_id = "{}.tokopedia".format(client.project)
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
try:
    dataset = client.get_dataset(dataset_id)  # Make an API request.
    print("Dataset {} already exists".format(dataset_id))
except NotFound: 
    dataset = client.create_dataset(dataset, timeout=30)

Dataset neo4jbusinessdev.tokopedia already exists


In [37]:
def clear_records(table_id): 
  sql = """
      DELETE FROM `%s`
      WHERE 1=1
  """ % (table_id)

  query = client.query(sql)
  print("started query as {}".format(query.job_id))
  query.result()


def load_table(table_name, schema, data_uri): 
    table_id = dataset_id + "." + table_name

    table = bigquery.Table(table_id, schema=schema)
    try:
        table = client.get_table(table)
        print("Table {} already exists".format(table))
    except NotFound:
        table = client.create_table(table)
        
    job_config = bigquery.LoadJobConfig(
      schema=schema,
      skip_leading_rows=1,
    )

    clear_records(table_id)

    load_job = client.load_table_from_uri(
      data_uri, table_id, job_config=job_config
    )

    load_job.result()

    destination_table = client.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))

### Category

In [34]:
load_table('category', [
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("name", "STRING"),
    ], 'gs://demo-tokopedia/similarity/import_node_category.csv')

started query as 026d961d-83b9-43b0-b2d4-172c495fdff0
Loaded 1 rows.


### SKU

In [38]:
load_table('sku', [
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("attributes", "STRING"),
        bigquery.SchemaField("brand", "STRING"),
        bigquery.SchemaField("colour", "STRING"),
        bigquery.SchemaField("serial", "STRING"),
    ], 'gs://demo-tokopedia/similarity/import_node_sku.csv')

started query as 3faf54f5-a608-4e74-98f4-37ccd3789aa9
Loaded 1000 rows.


### Keyword

In [39]:
load_table('keyword', [
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("keywords", "STRING"),
    ], 'gs://demo-tokopedia/similarity/import_node_keyword.csv')

started query as e80f4a54-2eca-4408-8072-002d323fde93
Loaded 4000 rows.


### Product

In [40]:
load_table('product', [
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("shop_id", "INTEGER"),
        bigquery.SchemaField("name", "STRING"),
    ], 'gs://demo-tokopedia/similarity/import_node_product.csv')

started query as 4acbcd5d-d9c4-4f3b-9662-cebe1084f1ef
Loaded 50000 rows.


### Attribute

In [41]:
load_table('attribute', [
        bigquery.SchemaField("id", "INTEGER"),
        bigquery.SchemaField("type", "STRING"),
        bigquery.SchemaField("value", "STRING"),
    ], 'gs://demo-tokopedia/similarity/import_node_attribute.csv')

started query as 2e968d85-d3fc-4752-97f3-59582c453b5a
Loaded 200 rows.


### IS_CATEGORY

In [42]:
load_table('rel_is_category', [
        bigquery.SchemaField("category_id", "INTEGER"),
        bigquery.SchemaField("sku_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_IS_CATEGORY.csv')


started query as dbb29928-8e4d-49c3-aea7-ba912f48d649
Loaded 1000 rows.


### IS_SKU

In [43]:
load_table('rel_is_sku', [
        bigquery.SchemaField("product_id", "INTEGER"),
        bigquery.SchemaField("sku_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_IS_SKU.csv')


started query as 3a0836d0-93d6-43a2-a9a1-6d12053d97ec
Loaded 50000 rows.


### WITH_KEYWORD

In [44]:
load_table('rel_with_keyword', [
        bigquery.SchemaField("keyword_id", "INTEGER"),
        bigquery.SchemaField("sku_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_WITH_KEYWORD.csv')


started query as eb30731f-b615-4035-a684-ad00406af9c5
Loaded 4000 rows.


### HOT_SALE

In [45]:
load_table('rel_hot_sale', [
        bigquery.SchemaField("product_id", "INTEGER"),
        bigquery.SchemaField("sku_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_HOT_SALE.csv')


started query as 38e8e781-6ab0-44ae-983d-daa913dd2aca
Loaded 4931 rows.


### SUPPLEMENT_WITH

In [46]:
load_table('rel_supplement_with', [
        bigquery.SchemaField("from_sku_id", "INTEGER"),
        bigquery.SchemaField("to_sku_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_SUPPLEMENT_WITH.csv')


started query as c2022d2a-ee7c-4d4d-adee-0880ea355e60
Loaded 500 rows.


### LOW_PRICE

In [47]:
load_table('rel_low_price', [
        bigquery.SchemaField("sku_id", "INTEGER"),
        bigquery.SchemaField("product_id", "INTEGER"),
    ], 'gs://demo-tokopedia/similarity/import_relation_LOW_PRICE.csv')

started query as bf23b510-6a59-4df5-8424-45748136b5f2
Loaded 5152 rows.


# Working with Neo4j
You'll need to enter the credentials from your Neo4j instance below.  You can get these by running the command ":server connect" in the Neo4j Browser.  The default DB_USER and DB_NAME are always neo4j.

In [50]:
# Edit these variables!
DB_URL = "neo4j+s://databases.neo4j.io:7687"
DB_PASS = ""

# You can leave this default
DB_USER = 'neo4j'

In [51]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS), aura_ds=True)

# Data Import

## Create Constraints

In [52]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT childCategoryIdConstraint IF NOT EXISTS FOR (c:Category) REQUIRE c.id IS UNIQUE
  """
)
display(result)

""


In [53]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT attributeIdConstraint IF NOT EXISTS FOR (a:Attribute) REQUIRE a.id IS UNIQUE
  """
)
display(result)

""


In [54]:
result = gds.run_cypher(
  """
    CREATE CONSTRAINT productIdConstraint IF NOT EXISTS FOR (p:Product) REQUIRE p.id IS UNIQUE
  """
)
display(result)

""


In [55]:
result = gds.run_cypher(
  """
    SHOW CONSTRAINTS YIELD id, name, type, entityType, labelsOrTypes, properties, ownedIndexId;
  """
)
display(result)

,id,name,type,entityType,labelsOrTypes,properties,ownedIndexId
0,6,attributeIdConstraint,UNIQUENESS,NODE,[Attribute],[id],5
1,4,childCategoryIdConstraint,UNIQUENESS,NODE,[Category],[id],3
2,8,productIdConstraint,UNIQUENESS,NODE,[Product],[id],7


## Load Data from Big Query

Create Nodes

In [ ]:
result = gds.run_cypher(
  """
    CALL apoc.periodic.iterate(
      'CALL apoc.load.jdbc("jdbc:mysql://localhost:3306/northwind?user=root","company")',
      'MERGE (n:Category {id:id,name:name})',
      { batchSize:10000, parallel:true})
    RETURN batches, total
    
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_sku.csv' AS line WITH line
    MERGE (n:SKU {id:line.id,name:line.name,attributes:line.attributes,brand:toInteger(line.brand),colour:toInteger(line.colour),serial:toInteger(line.serial)})
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_keyword.csv' AS line WITH line
    MERGE (n:Keyword {id:line.id,keywords:line.keywords})
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_product.csv' AS line WITH line
    MERGE (n:Product {id:line.id,shop_id:line.shop_id,name:line.name})
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_node_attribute.csv' AS line WITH line
    MERGE (n:Attribute {id:line.id,type:line.type,value:line.value})
  """
)
display(result)

""


Create Relationships

In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_IS_CATEGORY.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Category {id:line.category_id})
    CREATE (from)-[:IS_CATEGORY]->(to)
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_IS_SKU.csv' AS line 
    WITH line
    MATCH (from:Product {id:line.product_id}), (to:SKU {id:line.sku_id})
    CREATE (from)-[:IS_SKU]->(to)
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_WITH_KEYWORD.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Keyword {id:line.keyword_id})
    CREATE (from)-[:WITH_KEYWORD]->(to)
  """
)
display(result)

""


In [ ]:
result = gds.run_cypher(
  """
    LOAD CSV WITH HEADERS 
    FROM 'https://raw.githubusercontent.com/ezhilvendhan/ecommerce-demo-gds-vertex-ai/main/data/import_relation_HOT_SALE.csv' AS line 
    WITH line
    MATCH (from:SKU {id:line.sku_id}), (to:Product {id:line.product_id})
    CREATE (from)-[:HOT_SALE]->(to)
  """
)
display(result)

""


# Upload to Google Cloud Storage
Now we can upload our data sets to our bucket.

In [ ]:
from google.cloud import storage
client = storage.Client()

Run the code below to create bucket, if needed. If the bucket exists, you get an error

In [ ]:
bucket = client.bucket(STORAGE_BUCKET)
if(client.get_bucket(bucket) is None):
  bucket.location=REGION
  client.create_bucket(bucket)

In [ ]:
filename='raw.csv'
upload_path = os.path.join('similarity', filename)
blob = bucket.blob(upload_path)
blob.upload_from_filename(filename)

## Create Big Query Dataset & Table

In [ ]:
import pandas_gbq
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [ ]:

dataset_id = "{}.tokopedia".format(client.project)
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
try:
    dataset = client.get_dataset(dataset_id)  # Make an API request.
    print("Dataset {} already exists".format(dataset_id))
except NotFound:
    dataset = client.create_dataset(dataset, timeout=30)
schema = [						
    bigquery.SchemaField("a_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("a_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("a_colour", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("a_brand", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("a_serial", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("b_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("b_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("b_colour", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("b_brand", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("b_serial", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("is_similar", "BOOLEAN", mode="REQUIRED"),
]
table_id = dataset_id+".similar_sku"
table = bigquery.Table(table_id, schema=schema)
try:
    table = client.get_table(table)
    print("Table {} already exists".format(table))
except NotFound:
    table = client.create_table(table)

table_id

Dataset neo4jbusinessdev.tokopedia already exists
Table Table(TableReference(DatasetReference('neo4jbusinessdev', 'tokopedia'), 'similar_sku')) already exists


'neo4jbusinessdev.tokopedia.similar_sku'

In [ ]:
pandas_gbq.to_gbq(df, 'tokopedia.similar_sku', project_id=PROJECT_ID, if_exists='replace')


similar_sku


1it [00:04,  4.73s/it]


# Train a Model on GCP
We'll use the original features to train an AutoML model.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

# read from Cloud Storage
# dataset = aiplatform.TabularDataset.create(
#     display_name="similarity-raw",
#     gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'similarity', 'raw.csv'),
# )

# read from Big Query
dataset = aiplatform.TabularDataset.create(
    display_name="similarity-raw",
    bq_source="bq://"+table_id
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

Creating TabularDataset
Create TabularDataset backing LRO: projects/803648085855/locations/us-central1/datasets/4241823500983074816/operations/2143198370150219776
TabularDataset created. Resource name: projects/803648085855/locations/us-central1/datasets/4241823500983074816
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/803648085855/locations/us-central1/datasets/4241823500983074816')
	Dataset: "similarity-raw"
	name: "projects/803648085855/locations/us-central1/datasets/4241823500983074816"


In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='similarity-raw',
    optimization_prediction_type='classification'
)

In [ ]:
model = job.run(
    dataset=dataset, 
    target_column="is_similar", 
    training_fraction_split=0.8, 
    validation_fraction_split=0.1, 
    test_fraction_split=0.1, 
    model_display_name="similarity-raw", 
    disable_early_stopping=False, 
    budget_milli_node_hours=1000, 
)

No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['b_brand', 'a_colour', 'b_colour', 'a_id', 'a_brand', 'b_name', 'b_serial', 'a_serial', 'b_id', 'a_name'].
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3019595655308902400?project=803648085855
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/3019595655308902400 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/3019595655308902400 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/3019595655308902400 current state:
PipelineState.PIPELINE_STATE_RUNNING


1000 milli node hours, or one node hour, is the minimum budget that Vertex AI allows.  However, Vertex AI isn't respecting that budget currently.  This job will probably run for two and a half hours.  

We're going to move on while that runs.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.